In [80]:
#pip install xgboost

Install and Importing files for further analysis

In [81]:
import pandas as pd
import numpy as np
from importlib.machinery import SourceFileLoader
dataprep = SourceFileLoader('sets', 'D:/UTS_MDSI/ADSI/Part A/src/data/prepare.py').load_module()
sets = SourceFileLoader('sets', 'D:/UTS_MDSI/ADSI/Part A/src/data/sets.py').load_module()
base = SourceFileLoader('base', 'D:/UTS_MDSI/ADSI/Part A/src/models/null.py').load_module()
from IPython.display import display
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV,StratifiedKFold
import joblib as job
from sklearn.preprocessing import MinMaxScaler

Modelling Base and best model selection

In [82]:
def score_base(y_train_preds, y_train, y_val_preds, y_val, f1_average='weighted'):
    name = 'Base'
    model_scores = []
    t_acc = accuracy_score(y_train, y_train_preds)
    t_prec = precision_score(y_train, y_train_preds)
    t_rec = recall_score(y_train, y_train_preds)
    t_f1 = f1_score(y_train, y_train_preds, average=f1_average)
    #t_auc = roc_auc_score(y_t, clf.predict_proba(X_t)[:, 1])
    v_acc = accuracy_score(y_val, y_val_preds)
    v_prec = precision_score(y_val, y_val_preds)
    v_rec = recall_score(y_val, y_val_preds)
    v_f1 = f1_score(y_val, y_val_preds, average=f1_average)
    #v_auc = roc_auc_score(y_v, clf.predict_proba(X_v)[:, 1])
    model_scores.append([name, t_acc, t_prec, t_rec, t_f1, v_acc, v_prec, v_rec, v_f1])
    df_model_scores = pd.DataFrame (model_scores, columns = ['model','t_accuracy','t_precision','t_recall','t_F1','v_accuracy','v_precision','v_recall','v_F1'])
    display(df_model_scores)
    
def fit_score_models(models, X_t, y_t, X_v, y_v, dump_model="NO"):
    model_scores = []
    best_acc = 0
    i = 0
    for name, model in models.items():
        i = i+1;
        clf = model
        if dump_model == "YES":
            job.dump(clf, "D:/UTS_MDSI/ADSI/Part A/models/yatindra_" + name + ".joblib", compress=3)
        clf.fit(X_t, y_t)
        t_preds = clf.predict(X_t)
        t_acc = accuracy_score(y_t, t_preds)
        if i == 1:
            best_acc = t_acc
            best_clf = clf
        else:
            if t_acc > best_acc:
                best_acc = t_acc
                best_clf = clf            
        t_prec = precision_score(y_t, t_preds)
        t_rec = recall_score(y_t, t_preds)
        t_f1 = f1_score(y_t, t_preds)
        t_auc = roc_auc_score(y_t, clf.predict_proba(X_t)[:, 1])
        v_preds = clf.predict(X_v)
        v_acc = accuracy_score(y_v, v_preds)
        v_prec = precision_score(y_v, v_preds)
        v_rec = recall_score(y_v, v_preds)
        v_f1 = f1_score(y_v, v_preds)
        v_auc = roc_auc_score(y_v, clf.predict_proba(X_v)[:, 1])
        model_scores.append([name, t_acc, t_prec, t_rec, t_f1, t_auc, v_acc, v_prec, v_rec, v_f1, v_auc])
    df_model_scores = pd.DataFrame (model_scores, columns = ['model','t_accuracy','t_precision','t_recall','t_F1','t_roc auc','v_accuracy','v_precision','v_recall','v_F1','v_roc auc'])
    display(df_model_scores)
    return best_clf

Load data sets cleaned

In [83]:
X_train, y_train, X_val, y_val, X_test, y_test = sets.load_sets(path='D:/UTS_MDSI/ADSI/Part A/processed/New_test/')

Baseline Accuracy for Training and Validation datasets

In [84]:
base_model = base.NullModel(target_type="classification")
y_base_train_preds = base_model.fit_predict(y_train)
y_base_val_preds = base_model.fit_predict(y_val)
score_base(y_base_train_preds, y_train, y_base_val_preds, y_val)

D:/UTS_MDSI/ADSI/Part A/src/models/null.py:45: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  self.pred_value = mode(y)[0][0]


,model,t_accuracy,t_precision,t_recall,t_F1,v_accuracy,v_precision,v_recall,v_F1
0,Base,0.830263,0.830263,1.0,0.753265,0.843829,0.843829,1.0,0.772357


Training multiple models to understand the best fit model

In [85]:
models_to_fit = {"Logistic Regression": LogisticRegression(random_state=8, solver='liblinear'),
                 "KNN Euclidian": KNeighborsClassifier(metric='euclidean'),
                 "KNN Manhattan": KNeighborsClassifier(metric='manhattan'),
                 "XGBoost": xgb.XGBClassifier(random_state=8, use_label_encoder=False)}
clf1 = fit_score_models (models_to_fit, X_train, y_train, X_val, y_val)

C:\Users\Yatindra\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1

,model,t_accuracy,t_precision,t_recall,t_F1,t_roc auc,v_accuracy,v_precision,v_recall,v_F1,v_roc auc
0,Logistic Regression,0.830420,0.832594,0.996017,0.907003,0.700216,0.840050,0.844543,0.993284,0.912894,0.726378
1,KNN Euclidian,0.848213,0.859263,0.977243,0.914463,0.850032,0.824307,0.856279,0.951493,0.901379,0.589391
2,KNN Manhattan,0.850417,0.862364,0.975536,0.915465,0.856721,0.814861,0.849599,0.948507,0.896333,0.592689
3,XGBoost,0.970241,0.965397,1.000000,0.982394,0.999229,0.828715,0.853175,0.962687,0.904628,0.662915


Performing grid search for XGBoost to understand which hyperparameters would result in best performance of the model

In [86]:
# grid search
clf1 = xgb.XGBClassifier(use_label_encoder=False)
n_estimators = [100, 200, 300, 400, 500]
learning_rate = [0.0001, 0.001, 0.01, 0.1]
param_grid = dict(learning_rate=learning_rate, n_estimators=n_estimators)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=8)
grid_search = GridSearchCV(clf1, param_grid, scoring="accuracy", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)
# Print best score and parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

C:\Users\Yatindra\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\Yatindra\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Best: 0.831523 using {'learning_rate': 0.01, 'n_estimators': 100}


Train XGBoost with best Hyperparameter

In [87]:
models_to_fit = {"XGBoost": xgb.XGBClassifier(random_state=8, use_label_encoder=False, learning_rate=0.01, n_estimators=300)}
clf2 = fit_score_models (models_to_fit, X_train, y_train, X_val, y_val, "YES")

,model,t_accuracy,t_precision,t_recall,t_F1,t_roc auc,v_accuracy,v_precision,v_recall,v_F1,v_roc auc
0,XGBoost,0.848843,0.847318,0.997724,0.916391,0.877157,0.842569,0.84802,0.991045,0.913971,0.696073


Output

In [88]:
df_test = pd.read_csv('D:/UTS_MDSI/ADSI/Part A/raw/2022_test.csv')


In [99]:
df_cleaned = df_test.copy()
df_cleaned.head()

,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV
0,0,56,9.1,4.0,1.6,3.7,43.7,0.1,0.3,7.3,0.7,1.2,63.4,1.2,0.8,1.7,0.4,0.2,0.3,0.8
1,1,43,19.3,10.1,3.7,8.1,46.0,0.6,1.7,35.1,1.8,2.5,75.3,0.5,0.9,1.5,3.5,0.6,0.0,1.8
2,2,82,33.9,11.3,4.9,10.6,45.6,0.5,1.9,44.8,1.8,2.7,71.2,1.3,3.3,4.5,2.5,1.3,0.3,2.0
3,3,86,44.7,18.8,6.8,15.9,42.9,0.5,1.8,13.5,4.5,6.3,70.9,1.5,3.2,5.0,4.1,0.9,0.1,3.6
4,4,58,12.3,4.7,1.6,4.0,40.0,0.5,1.7,38.7,1.1,1.3,76.9,0.2,0.6,0.9,1.5,0.5,-0.4,0.9


In [105]:
#df_cleaned = dataprep.remove_invalid_rows(df_cleaned, ['GP','FT%'])
X_test = df_cleaned.copy()
X_test = dataprep.drop_features(X_test, ['Id','3P%']) #,'3PA','3P%'
X_test.head()
#print(X_test)
X_test = dataprep.scale_features(X_test, MinMaxScaler(), None)
X_test.info()
test_probs = clf2.predict_proba(X_test)[:, 1]
df_output = pd.DataFrame({'Id': df_cleaned['Id'], 'TARGET_5Yrs': test_probs});
df_output.shape
df_output.to_csv("D:/UTS_MDSI/ADSI/Part A/submission/Yatindra-_xgboost1.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3799 entries, 0 to 3798
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   GP       3799 non-null   float64
 1   MIN      3799 non-null   float64
 2   PTS      3799 non-null   float64
 3   FGM      3799 non-null   float64
 4   FGA      3799 non-null   float64
 5   FG%      3799 non-null   float64
 6   3P Made  3799 non-null   float64
 7   3PA      3799 non-null   float64
 8   FTM      3799 non-null   float64
 9   FTA      3799 non-null   float64
 10  FT%      3799 non-null   float64
 11  OREB     3799 non-null   float64
 12  DREB     3799 non-null   float64
 13  REB      3799 non-null   float64
 14  AST      3799 non-null   float64
 15  STL      3799 non-null   float64
 16  BLK      3799 non-null   float64
 17  TOV      3799 non-null   float64
dtypes: float64(18)
memory usage: 534.4 KB
